In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation/Predictions/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read GDSC cell line details table

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (DNA_REPi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (DNA_REPi)

GDSCE = pd.read_csv("GDSC_exprs.z.DNA_REPi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC mutation dataset (DNA_REPi)

os.chdir(dataset_dir + "/mutations/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations


In [9]:
# read GDSC mutation dataset (DNA_REPi)

GDSCM = pd.read_csv("GDSC_mutations.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCM.drop_duplicates(keep='last')
GDSCM = pd.DataFrame.transpose(GDSCM)
GDSCM = GDSCM.loc[:, ~GDSCM.columns.duplicated()]

GDSCM.head()

,143872,728577,2,2050,10243,79365,256006,4609,118788,113675,...,26517,28972,53917,283507,514,339210,64981,7335,5739,83442
683665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# change directory to read GDSC response dataset (DNA_REPi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [11]:
# change directory to read GDSC response dataset (DNA_REPi)

GDSCR = pd.read_csv("GDSC_response.DNA_REPi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1
683665,0,-4.40897202584936,Gemcitabine,1,1,1
683665,0,-1.59752443691324,Mitomycin C,1,1,1


In [12]:
# print count of cell lines for each drug (DNA_REPi)

GDSCR["drug"].value_counts()

drug
Etoposide         775
SN-38             771
Mitomycin C       767
Doxorubicin       766
5-Fluorouracil    764
Gemcitabine       761
Bleomycin         759
Temozolomide      752
Cisplatin         738
Methotrexate      738
Camptothecin      737
Cytarabine        735
Pyrimethamine     372
Name: count, dtype: int64

In [13]:
# change directory to read TCGA expression dataset homogenized with GDSC expression dataset (Temozolomide)

os.chdir(dataset_dir + "/exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [14]:
# read TCGA expression dataset homogenized with GDSC expression dataset (Temozolomide)

TCGAEtemozolomide = pd.read_csv("TCGA_exprs.Temozolomide.eb_with.GDSC_exprs.Temozolomide.tsv",
                   sep = "\t", index_col=0, decimal = ",")
TCGAEtemozolomide = pd.DataFrame.transpose(TCGAEtemozolomide)
TCGAEtemozolomide.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100188893,100190940,100190949,100191040,100192386,100268168,100271715,100287718,100288778,100289635
TCGA-DB-A4XB-01A,3.01411500679127,3.21429233402942,6.4528333134497,2.95628179386887,2.11182452311578,2.96154806728763,8.49988388272576,2.62293993624751,8.9387140767194,4.15154137558802,...,9.17318432966048,2.64970810303467,2.74560040876787,3.32036177484863,3.45134028609217,2.84865221646127,3.28055555566557,2.72651224971197,5.5924078445875,4.99760858931826
TCGA-DB-A4XD-01A,4.64454942106989,4.2132521109463,4.98533650210385,2.49132421622367,2.26592937938765,2.96154806728763,8.11985458578506,2.36514294812994,8.56015015827888,4.34906181922056,...,9.00189428232398,2.62642389297931,2.74560040876787,3.09176896498447,3.58343753923839,2.44226819244948,2.53952138693823,2.72651224971197,4.44402883666404,4.2826105962019
TCGA-DB-A4XF-01A,3.81691719009577,4.61811398228358,5.61623849668032,4.35255263922777,5.35342357746377,2.96154806728763,7.70766074541521,2.51636328208313,8.25973564321625,4.07335851817075,...,8.87919475426985,2.80493720046034,2.74560040876787,3.00804657882649,3.97492729783926,2.72074746077948,2.68867433708246,2.72651224971197,4.42517105663064,4.56343673951182


In [15]:
# change directory to read TCGA mutation dataset (Temozolomide)

os.chdir(dataset_dir + "mutations")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations'

In [16]:
# read TCGA mutation dataset (Temozolomide)

TCGAMtemozolomide = pd.read_csv("TCGA_mutations.Temozolomide.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGAMtemozolomide.drop_duplicates(keep='last')
TCGAMtemozolomide = pd.DataFrame.transpose(TCGAMtemozolomide)
TCGAMtemozolomide = TCGAMtemozolomide.loc[:, ~TCGAMtemozolomide.columns.duplicated()]

TCGAMtemozolomide.head(3)

,2312,220042,23054,6938,7580,10656,546,4648,494513,9912,...,89,196913,3139,6747,5544,4747,23148,407835,81611,2919
TCGA-DB-A4XB-01A,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-DB-A4XD-01A,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-DB-A4XF-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# variance threshold for GDSC expression dataset (DNA_REPi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [18]:
# fill NA values and binarize GDSC mutation dataset (DNA_REPi)

GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1

In [19]:
# select shared genes between GDSC and TCGA expression and mutation datasets 

ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(TCGAEtemozolomide.columns)
ls = ls.intersection(TCGAMtemozolomide.columns)

ls = pd.unique(ls)

print(f"GDSC and TCGA # of common genes: {ls.shape[0]}")

GDSC and TCGA # of common genes: 16584


In [20]:
# select shared samples between GDSC expression, mutation, and response datasets (DNA_REPi)

ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 801


In [21]:
# subset shared genes and samples in GDSC expression, mutation, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCM = GDSCM.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCM.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

GDSC # of common samples and genes (Expression): (801, 16584)
GDSC # of common samples and genes (Mutation): (801, 16584)
GDSC # of samples (Response): 9435


In [22]:
# select shared samples between TCGA expression and mutation datasets (Temozolomide) 

ls3 = TCGAEtemozolomide.index.intersection(TCGAMtemozolomide.index)

print(f"TCGA # of common samples (Temozolomide): {ls3.shape[0]}")

TCGA # of common samples (Temozolomide): 90


In [23]:
# select shared genes and samples between TCGA expression and mutation datasets (Temozolomide)

TCGAEtemozolomide = TCGAEtemozolomide.loc[ls3, ls]
TCGAMtemozolomide = TCGAMtemozolomide.loc[ls3, ls]

print(f"TCGA # of common samples and genes for Expression (Temozolomide): {TCGAEtemozolomide.shape}")
print(f"TCGA # of common samples and genes for Mutation (Temozolomide): {TCGAMtemozolomide.shape}\n")

TCGA # of common samples and genes for Expression (Temozolomide): (90, 16584)
TCGA # of common samples and genes for Mutation (Temozolomide): (90, 16584)



In [24]:
# change directory to read DEGs (DNA_REPi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi


In [25]:
# read DEGs (DNA_REPi)

DEGs_filtered_data = pd.read_excel("DNA_REPi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors\n")

DEGs_filtered_data.head()

There are 858 DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors



,Gene Symbol,Frequency
0,SLFN11,9
1,NCKAP1L,8
2,SDC4,7
3,KRT18,7
4,S100A14,7


In [26]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id.shape[0]

1 input query terms found dup hits:	[('CLECL1P', 2)]


858

In [27]:
# assign GDSC data to new variables

exprs_z = GDSCE
mut = GDSCM
responses = GDSCR

In [28]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1


In [29]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'5-Fluorouracil',
 'Bleomycin',
 'Camptothecin',
 'Cisplatin',
 'Cytarabine',
 'Doxorubicin',
 'Etoposide',
 'Gemcitabine',
 'Methotrexate',
 'Mitomycin C',
 'Pyrimethamine',
 'SN-38',
 'Temozolomide'}

In [30]:
# convert Entrez gene IDs to integer

responses.index = responses.index.astype(int)
responses.index

Index([ 683665,  683665,  683665,  683665,  683665,  683665,  683665,  683665,
        683665,  683665,
       ...
       1659818, 1659818, 1659818, 1659818, 1659819, 1659819, 1659819, 1659819,
       1659819,  687800],
      dtype='int64', length=9435)

In [31]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
         responses,
         left_index = True, 
         right_index=True,
         how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
#dat.head()

GDSC\nTissue descriptor 1
lung_NSCLC           1037
urogenital_system     926
leukemia              829
lymphoma              769
aero_dig_tract        727
nervous_system        607
lung_SCLC             569
skin                  560
breast                524
digestive_system      466
large_intestine       447
bone                  421
neuroblastoma         332
pancreas              294
kidney                227
soft_tissue           191
myeloma               182
lung                  174
thyroid               153
Name: count, dtype: int64

In [32]:
# list count of cell lines as to tissues treated with temozolomide

filter = (dat["drug"] == "temozolomide")
dat.loc[filter,"GDSC\nTissue descriptor 1"].value_counts()

Series([], Name: count, dtype: int64)

In [33]:
# convert Entrez gene IDs to string

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '1659818', '1659818', '1659818', '1659818', '1659819', '1659819',
       '1659819', '1659819', '1659819', '687800'],
      dtype='object', length=9435)

In [34]:
# filter GDSC expression and mutation datasets (DNA_REPi) as to drugs
# subset selected DEGs

expression_zscores = []
mutations = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    m = mut.loc[samples, :]
    expression_zscores.append(e_z)
    mutations.append(m)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCMv2 = pd.concat(mutations, axis=0)
GDSCRv2 = responses

ls4 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls5 = GDSCEv2.index.intersection(GDSCMv2.index)

GDSCEv2 = GDSCEv2.loc[ls5, ls4]
GDSCMv2 = GDSCMv2.loc[ls5, ls4]
GDSCRv2 = GDSCRv2.loc[ls5, :]

TCGAEtemozolomide = TCGAEtemozolomide.loc[:,ls4]
TCGAMtemozolomide = TCGAMtemozolomide.loc[:,ls4]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCMv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"TCGA # of common samples and genes for temozolomide (Expression): {TCGAEtemozolomide.shape}")
print(f"TCGA # of common samples and genes for temozolomide (Mutation): {TCGAMtemozolomide.shape}\n")

GDSC # of common samples and genes (Expression): (9435, 799)
GDSC # of common samples and genes (Mutation): (9435, 799)
GDSC # of common samples (Response): 9435

TCGA # of common samples and genes for temozolomide (Expression): (90, 799)
TCGA # of common samples and genes for temozolomide (Mutation): (90, 799)



In [35]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 1 0 0]


In [36]:
# change directory to read TCGA response dataset (Temozolomide)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [37]:
# read TCGA response dataset (Temozolomide)

TCGARtemozolomide = pd.read_csv("TCGA_response.Temozolomide.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
TCGARtemozolomide.dropna(inplace=True)
TCGARtemozolomide.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
TCGARtemozolomide["response"] = TCGARtemozolomide.loc[:, "response"].apply(lambda x: d[x])
Ytstemozolomide = TCGARtemozolomide["response"].values    

print(f"There are {Ytstemozolomide.shape[0]} samples with response data in the TCGA (Temozolomide)\n")
Ytstemozolomide

There are 90 samples with response data in the TCGA (Temozolomide)



array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0])

In [38]:
# TCGA cohort table with TCGA response dataset (Temozolomide)

dat_tcga = TCGARtemozolomide["cohort"]

dat_tcga.value_counts()

cohort
LGG    90
Name: count, dtype: int64

In [39]:
# filter LGG cohort samples (Temozolomide)

filter = (TCGARtemozolomide["cohort"] == "LGG")
tcga_lgg_samples = dat_tcga.loc[filter,].index
tcga_lgg_samples

Index(['TCGA-DB-A4XB-01A', 'TCGA-DB-A4XD-01A', 'TCGA-DB-A4XF-01A',
       'TCGA-DB-A4XG-01A', 'TCGA-DB-A64O-01A', 'TCGA-DB-A64P-01A',
       'TCGA-DB-A64V-01A', 'TCGA-DB-A64W-01A', 'TCGA-DB-A64X-01A',
       'TCGA-DB-A75K-01A', 'TCGA-DH-A66B-01A', 'TCGA-DH-A66D-01A',
       'TCGA-DH-A66G-01A', 'TCGA-DH-A7UR-01A', 'TCGA-DH-A7UT-01A',
       'TCGA-DH-A7UU-01A', 'TCGA-DH-A7UV-01A', 'TCGA-DU-8162-01A',
       'TCGA-DU-8163-01A', 'TCGA-DU-8165-01A', 'TCGA-DU-8166-01A',
       'TCGA-DU-8168-01A', 'TCGA-DU-A5TS-01A', 'TCGA-DU-A5TY-01A',
       'TCGA-DU-A6S6-01A', 'TCGA-DU-A6S7-01A', 'TCGA-DU-A76K-01A',
       'TCGA-DU-A76O-01A', 'TCGA-DU-A76R-01A', 'TCGA-DU-A7TA-01A',
       'TCGA-E1-A7YW-01A', 'TCGA-FG-6689-01A', 'TCGA-FG-7638-01B',
       'TCGA-FG-8189-01B', 'TCGA-FG-8191-01A', 'TCGA-FG-A4MW-01A',
       'TCGA-FG-A6J3-01A', 'TCGA-FG-A70Y-01A', 'TCGA-FG-A711-01A',
       'TCGA-FN-7833-01A', 'TCGA-HT-7884-01B', 'TCGA-HT-8114-01A',
       'TCGA-HT-8564-01A', 'TCGA-HT-A5RA-01A', 'TCGA-HT-A5RC-0

In [40]:
# filter responses of LGG cohort samples (Temozolomide)

Ytstemozolomide_lgg = TCGARtemozolomide.loc[tcga_lgg_samples,"response"].values
Ytstemozolomide_lgg

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0])

In [41]:
# count of sensitive and resistance LGG samples (Temozolomide)

TCGARtemozolomide.loc[tcga_lgg_samples,"response"].value_counts()

response
0    79
1    11
Name: count, dtype: int64

In [42]:
# define maximum iteration

max_iter = 50

In [51]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 256
hdm2 = 1024
rate1 = 0.3
rate2 = 0.4
rate3 = 0.6

In [52]:
# load pre-trained models and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)

X_testEtemozolomide = scalerGDSC.transform(TCGAEtemozolomide.values)    
X_testEtemozolomide_lgg = scalerGDSC.transform(TCGAEtemozolomide.loc[tcga_lgg_samples,].values)    

X_trainM = np.nan_to_num(GDSCMv2.values)
X_testMtemozolomide = np.nan_to_num(TCGAMtemozolomide.values.astype(np.float32))
X_testMtemozolomide_lgg = np.nan_to_num(TCGAMtemozolomide.loc[tcga_lgg_samples,].values.astype(np.float32))

TX_testEtemozolomide = torch.FloatTensor(X_testEtemozolomide)
TX_testEtemozolomide_lgg = torch.FloatTensor(X_testEtemozolomide_lgg)

TX_testMtemozolomide = torch.FloatTensor(X_testMtemozolomide.astype(np.float32))
TX_testMtemozolomide_lgg = torch.FloatTensor(X_testMtemozolomide_lgg.astype(np.float32))

ty_testEtemozolomide = torch.FloatTensor(Ytstemozolomide.astype(int))
ty_testEtemozolomide_lgg = torch.FloatTensor(Ytstemozolomide_lgg.astype(int))

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape

h_dim1 = hdm1
h_dim2 = hdm2
Z_in = h_dim1 + h_dim2 

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnM(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate3),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation.pt")
AutoencoderM = torch.load(models_dir + "Mut_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation.pt")

Clas = torch.load(models_dir + "Class_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation.pt")

AutoencoderE.eval()
AutoencoderM.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZMX = AutoencoderM(torch.FloatTensor(X_trainM))
ZTX = torch.cat((ZEX, ZMX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (TCGA_DNA_REPi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n")

ZETtemozolomide = AutoencoderE(TX_testEtemozolomide)
ZMTtemozolomide = AutoencoderM(TX_testMtemozolomide)
ZTTtemozolomide = torch.cat((ZETtemozolomide, ZMTtemozolomide), 1)
ZTTtemozolomide = F.normalize(ZTTtemozolomide, p=2, dim=0)
PredTtemozolomide = Clas(ZTTtemozolomide)
AUCTtemozolomide = roc_auc_score(Ytstemozolomide.astype(int), PredTtemozolomide.detach().numpy())
#print(f"TCGA - Test AUC (Temozolomide): {round(AUCTtemozolomide,2)}")
AUCTtemozolomide_aucpr = average_precision_score(Ytstemozolomide.astype(int), PredTtemozolomide.detach().numpy())
#print(f"TCGA - Test AUCPR (Temozolomide): {round(AUCTtemozolomide_aucpr,2)}\n")

ZETtemozolomide_lgg = AutoencoderE(TX_testEtemozolomide_lgg)
ZMTtemozolomide_lgg = AutoencoderM(TX_testMtemozolomide_lgg)
ZTTtemozolomide_lgg = torch.cat((ZETtemozolomide_lgg, ZMTtemozolomide_lgg), 1)
ZTTtemozolomide_lgg = F.normalize(ZTTtemozolomide_lgg, p=2, dim=0)
PredTtemozolomide_lgg = Clas(ZTTtemozolomide_lgg)
AUCTtemozolomide_lgg = roc_auc_score(Ytstemozolomide_lgg.astype(int), PredTtemozolomide_lgg.detach().numpy())
#print(f"TCGA - Test AUC (Temozolomide, LGG): {round(AUCTtemozolomide_lgg,2)}")
AUCTtemozolomide_lgg_aucpr = average_precision_score(Ytstemozolomide_lgg.astype(int), PredTtemozolomide_lgg.detach().numpy())
#print(f"TCGA - Test AUCPR (Temozolomide, LGG): {round(AUCTtemozolomide_lgg_aucpr,2)}\n")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTtemozolomide_finetuned = dict()
AUCTtemozolomide_lgg_finetuned = dict()
AUCTtemozolomide_aucpr_finetuned = dict()
AUCTtemozolomide_lgg_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{iter}.pt")
   AutoencoderM_finetuned = torch.load(models_dir + f"Finetuned_Models/Mut_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderM_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZMX = AutoencoderM_finetuned(torch.FloatTensor(X_trainM))
   ZTX = torch.cat((ZEX, ZMX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETtemozolomide = AutoencoderE_finetuned(TX_testEtemozolomide)
   ZMTtemozolomide = AutoencoderM_finetuned(TX_testMtemozolomide)
   ZTTtemozolomide = torch.cat((ZETtemozolomide, ZMTtemozolomide), 1)
   ZTTtemozolomide = F.normalize(ZTTtemozolomide, p=2, dim=0)
   PredTtemozolomide = Clas_finetuned(ZTTtemozolomide)
   AUCTtemozolomide_finetuned[iter] = roc_auc_score(Ytstemozolomide.astype(int), PredTtemozolomide.detach().numpy())
   AUCTtemozolomide_aucpr_finetuned[iter] = average_precision_score(Ytstemozolomide.astype(int), PredTtemozolomide.detach().numpy())

   ZETtemozolomide_lgg = AutoencoderE_finetuned(TX_testEtemozolomide_lgg)
   ZMTtemozolomide_lgg = AutoencoderM_finetuned(TX_testMtemozolomide_lgg)
   ZTTtemozolomide_lgg = torch.cat((ZETtemozolomide_lgg, ZMTtemozolomide_lgg), 1)
   ZTTtemozolomide_lgg = F.normalize(ZTTtemozolomide_lgg, p=2, dim=0)
   PredTtemozolomide_lgg = Clas_finetuned(ZTTtemozolomide_lgg)
   AUCTtemozolomide_lgg_finetuned[iter] = roc_auc_score(Ytstemozolomide_lgg.astype(int), PredTtemozolomide_lgg.detach().numpy())
   AUCTtemozolomide_lgg_aucpr_finetuned[iter] = average_precision_score(Ytstemozolomide_lgg.astype(int), PredTtemozolomide_lgg.detach().numpy())

max_key = max(AUCTtemozolomide_aucpr_finetuned, key=AUCTtemozolomide_aucpr_finetuned.get)
max_key_lgg = max(AUCTtemozolomide_lgg_aucpr_finetuned, key=AUCTtemozolomide_lgg_aucpr_finetuned.get)

print(f"GDSC - Training AUC (DNA_REPi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (DNA_REPi): {round(AUCTaucpr,2)}")
print(f"TCGA - Test AUC (Temozolomide): {round(AUCTtemozolomide,2)}")
print(f"TCGA - Test AUCPR (Temozolomide): {round(AUCTtemozolomide_aucpr,2)}")
print(f"TCGA - Test AUC (Temozolomide, LGG): {round(AUCTtemozolomide_lgg,2)}")
print(f"TCGA - Test AUCPR (Temozolomide, LGG): {round(AUCTtemozolomide_lgg_aucpr,2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"TCGA - Test AUC (Temozolomide, Finetuned): {round(AUCTtemozolomide_finetuned[max_key],2)}")
print(f"TCGA - Test AUCPR (Temozolomide, Finetuned): {round(AUCTtemozolomide_aucpr_finetuned[max_key],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_lgg],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_lgg],2)}")
print(f"TCGA - Test AUC (Temozolomide, LGG, Finetuned): {round(AUCTtemozolomide_lgg_finetuned[max_key_lgg],2)}")
print(f"TCGA - Test AUCPR (Temozolomide, LGG, Finetuned): {round(AUCTtemozolomide_lgg_aucpr_finetuned[max_key_lgg],2)}\n\n")

print("Models for maximum finetuning (Temozolomide)")
print(f"Exprs_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key}.pt")
print(f"Mut_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key}.pt")
print(f"Class_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key}.pt")

print("\nModels for maximum finetuning (Temozolomide, LGG)")
print(f"Exprs_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key_lgg}.pt")
print(f"Mut_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key_lgg}.pt")
print(f"Class_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_{max_key_lgg}.pt")

GDSC - Training AUC (DNA_REPi): 0.83
GDSC - Training AUCPR (DNA_REPi): 0.35
TCGA - Test AUC (Temozolomide): 0.62
TCGA - Test AUCPR (Temozolomide): 0.17
TCGA - Test AUC (Temozolomide, LGG): 0.62
TCGA - Test AUCPR (Temozolomide, LGG): 0.17

GDSC - Training AUC (DNA_REPi, Finetuned): 0.83
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.35
TCGA - Test AUC (Temozolomide, Finetuned): 0.69
TCGA - Test AUCPR (Temozolomide, Finetuned): 0.19

GDSC - Training AUC (DNA_REPi, Finetuned): 0.83
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.35
TCGA - Test AUC (Temozolomide, LGG, Finetuned): 0.69
TCGA - Test AUCPR (Temozolomide, LGG, Finetuned): 0.19


Models for maximum finetuning (Temozolomide)
Exprs_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_10.pt
Mut_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_10.pt
Class_Temozolomide_GDSC_TCGA_DNA_REPi_Expression_Mutation_Finetuned_10.pt

Models for maximum finetuning (Temozolomide, LGG)
Exprs_Temozolomide_GDSC_TCGA_DNA_REPi_Ex

In [45]:
# create a dataframe with the prediction results 

em = {"Data": ["Expr + Mut", "Expr + Mut (Fine-tuned)"],
      "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
      "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
      "AUC (TCGA)": [round(AUCTtemozolomide,2), round(AUCTtemozolomide_finetuned[max_key],2)],
      "AUCPR (TCGA)":  [round(AUCTtemozolomide_aucpr,2), round(AUCTtemozolomide_aucpr_finetuned[max_key],2)],
      "AUC (TCGA-LGG)":  [round(AUCTtemozolomide_lgg,2), round(AUCTtemozolomide_lgg_finetuned[max_key_lgg],2)],
      "AUCPR (TCGA-LGG)": [round(AUCTtemozolomide_lgg_aucpr,2), round(AUCTtemozolomide_lgg_aucpr_finetuned[max_key_lgg],2)],
      "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
      "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
em_dataframe = pd.DataFrame.from_dict(em, orient='index').transpose() 

em_dataframe.to_csv(save_results_to + "GDSC_TCGA_Expression_Mutation_DNA_REPi_Temozolomide.tsv",
                    sep = "\t",
                    index = False)

em_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (TCGA),AUCPR (TCGA),AUC (TCGA-LGG),AUCPR (TCGA-LGG),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + Mut,0.83,0.35,0.62,0.17,0.62,0.17,9435,799
1,Expr + Mut (Fine-tuned),0.83,0.35,0.69,0.19,0.69,0.19,9435,799


In [46]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=799, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
)

In [47]:
# show mutation layer parameters

AutoencoderM

AEM(
  (EnM): Sequential(
    (0): Linear(in_features=799, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
)

In [48]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=1280, out_features=1, bias=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Sigmoid()
  )
)

In [49]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation/Predictions


In [50]:
# save predictions

file = open('GDSC_TCGA_DEGs - TCGA_DNA_REPi_Temozolomide Predictions (Expression and Mutation).txt', 'w')
file.write(f"GDSC Training (EM) AUC (TCGA_DNA_REPi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EM) AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n\n")

file.write(f"TCGA Test (EM) AUC (Temozolomide): {round(AUCTtemozolomide,2)}\n")
file.write(f"TCGA Test (EM) AUCPR (Temozolomide): {round(AUCTtemozolomide_aucpr,2)}\n\n")

file.close()